In [126]:
import importlib
import sys
import DataGenerator
import ShipDetection
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Input
from keras.optimizers import Adam
import pandas as pd
import glob

In [127]:
imgWidth = 768
imgHeight = 768
batch_size = 32
trainDir = "data/train_v2/"

In [128]:
trainPaths = glob.glob(trainDir + "*.jpg")

In [140]:
train_ship_segmentations_v2 = pd.read_csv("train_ship_segmentations_v2.csv")
train_ship_segmentations_v2.set_index("ImageId", inplace=True)

In [165]:
importlib.reload(DataGenerator)
importlib.reload(ShipDetection)

<module 'ShipDetection' from 'D:\\AdankoProject\\Project\\AirbusAhipDetectionTA\\ShipDetection.py'>

In [166]:
training_generator = DataGenerator.DataGenerator(trainPaths, train_ship_segmentations_v2)
validation_generator = DataGenerator.DataGenerator(trainPaths, train_ship_segmentations_v2)

In [149]:
inputImg = Input((imgWidth, imgHeight, 3), name='img')
model = ShipDetection.get_unet(inputImg)
model.compile(optimizer=Adam(), loss=ShipDetection.dice_coef_loss)

In [150]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 768, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_115 (Conv2D)            (None, 768, 768, 16  448         ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 batch_normalization_109 (Batch  (None, 768, 768, 16  64         ['conv2d_115[0][0]']             
 Normalization)                 )                                                           

In [151]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [168]:
results = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6,
                    epochs=50,
                    callbacks=callbacks
                    )

C:\Users\adanko\AppData\Local\Temp\ipykernel_2212\4131744744.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model.fit_generator(generator=training_generator,


Epoch 1/50


NotImplementedError: in user code:

    File "D:\AdankoProject\python\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "D:\AdankoProject\Project\AirbusAhipDetectionTA\ShipDetection.py", line 36, in dice_coef_loss  *
        return 1-dice_coef(y_true, y_pred)
    File "D:\AdankoProject\Project\AirbusAhipDetectionTA\ShipDetection.py", line 31, in dice_coef  *
        numerator = 2. * np.sum(y_pred * y_true, axes)
    File "<__array_function__ internals>", line 180, in sum  **
        
    File "D:\AdankoProject\python\lib\site-packages\numpy\core\fromnumeric.py", line 2298, in sum
        return _wrapreduction(a, np.add, 'sum', axis, dtype, out, keepdims=keepdims,
    File "D:\AdankoProject\python\lib\site-packages\numpy\core\fromnumeric.py", line 86, in _wrapreduction
        return ufunc.reduce(obj, axis, dtype, out, **passkwargs)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (dice_coef_lo